# Imports

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
import numpy as np
from lightfm.cross_validation import random_train_test_split
import os
from scipy.sparse import csr_matrix

# Load Data

In [12]:
data = pd.read_csv("./fnb_datav2.csv")

# Remove 
data = data.drop(columns = ["page", "tod"])
data.head(20)

,idcol,interaction,int_date,item,item_type,item_descrip,segment,beh_segment,active_ind
0,755,DISPLAY,17JAN2023,NONE,ALL,NaN,segment3,B01,Semi Active
1,4521,DISPLAY,27FEB2023,NONE,ALL,NaN,segment1,B07,Semi Active
2,4521,DISPLAY,18FEB2023,NONE,ALL,NaN,segment1,B07,Semi Active
3,4521,DISPLAY,30JAN2023,NONE,ALL,NaN,segment1,B07,Semi Active
4,4521,CLICK,05FEB2023,IBAB,INSURE,GENERIC MESSAGE,segment1,B07,Semi Active
5,4521,CHECKOUT,05FEB2023,IBAB,INSURE,GENERIC MESSAGE,segment1,B07,Semi Active
6,6145,DISPLAY,26FEB2023,NONE,ALL,NaN,segment3,B01,Cold Start
7,6145,DISPLAY,27JAN2023,NONE,ALL,NaN,segment3,B01,Cold Start
8,6145,DISPLAY,10FEB2023,NONE,ALL,NaN,segment3,B01,Cold Start
9,6145,DISPLAY,10JAN2023,NONE,ALL,NaN,segment3,B01,Cold Start


In [8]:
data.nunique()

idcol           84375
interaction         3
int_date           88
item              104
item_type           7
item_descrip      100
segment             4
beh_segment        50
active_ind          3
dtype: int64

# Add Features:

In [14]:
# Give scores to the interactions, and then drop the interaction column:L
interaction_scores = {
    'CLICK': 1,
    'CHECKOUT': 2
}

data['interaction_scores'] = data['interaction'].map(interaction_scores).fillna(0).astype(int)

data.drop(columns = ["interaction"])
data.head()

,idcol,interaction,int_date,item,item_type,item_descrip,segment,beh_segment,active_ind,interaction_scores
0,755,DISPLAY,17JAN2023,NONE,ALL,NaN,segment3,B01,Semi Active,0
1,4521,DISPLAY,27FEB2023,NONE,ALL,NaN,segment1,B07,Semi Active,0
2,4521,DISPLAY,18FEB2023,NONE,ALL,NaN,segment1,B07,Semi Active,0
3,4521,DISPLAY,30JAN2023,NONE,ALL,NaN,segment1,B07,Semi Active,0
4,4521,CLICK,05FEB2023,IBAB,INSURE,GENERIC MESSAGE,segment1,B07,Semi Active,1


In [ ]:
# Define the user columns, item columns and interaction columns:
u_cols = ["idcol", "segment", "beh_segment", "active_ind"]
item_cols = ["item", "item_type", "iteM_descrip"]
interact_cols = ["interaction", ""]